In [1]:
import os
HOME = os.getcwd()
HOME = os.chdir('..')
HOME = os.getcwd()

import numpy as np
import pandas as pd
import ee #pip install earthengine-api
import geemap
import utils.EE_funcs as EE_funcs
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import concurrent.futures as cf
import pyarrow as pa
import pyarrow.parquet as pq
import pickle as pkl
ee.Authenticate()
ee.Initialize()
import warnings
import boto3
warnings.filterwarnings("ignore")

In [54]:
# Function to add a 'day' property to each image ---
# This property will be used for grouping
def add_day_property(image):
    # Get the start time of the image
    date = ee.Date(image.get('system:time_start'))
    # Format it to YYYY-MM-dd to represent the day
    day_string = date.format('YYYY-MM-dd')
    # Return the image with the new 'day' property
    return image.set('day', day_string)


def sum_daily_precipitation(day_str):
    day_start = ee.Date(day_str)
    day_end = day_start.advance(1, 'day')
    daily_hourly_images = hourly_precip.filterDate(day_start, day_end)

    # Get a reference projection for consistent zero images
    try:
        ref_proj = hourly_precip.first().projection()
    except Exception:
        ref_proj = ee.Projection('EPSG:4326').atScale(scale)

    # Prepare a zero image
    zero_image = ee.Image.constant(0).rename('total_precipitation').unmask(0) \
                         .reproject(ref_proj) \
                         .clip(aoi_bbox)

    # Use ee.Algorithms.If to return sum or zero image
    daily_count = daily_hourly_images.size()
    result_image = ee.Algorithms.If(
        daily_count.eq(0),
        zero_image,
        daily_hourly_images.sum().clip(aoi_bbox).unmask(0)
    )
    result_image = ee.Image(result_image).set('system:time_start', day_start.millis()).set('day', day_str)
    return result_image


def add_month_property(image):
    date = ee.Date(image.get('system:time_start'))
    month_string = date.format('YYYY-MM')
    return image.set('month', month_string)

In [49]:
PrecipProduct = 'NASA/NLDAS/FORA0125_H002'
var = 'total_precipitation'


WYs = [2013]

region='2013'
output_res=1000
#  #ASO file path
aso_swe_files_folder_path = f"{HOME}/data/ASO/{region}/{output_res}M_SWE_parquet/"

#make directory for data 
Precippath = f"{HOME}/data/Precipitation/{region}/{output_res}M_NLDAS_Precip/sites"

if not os.path.exists(Precippath):
    os.makedirs(Precippath, exist_ok=True)

#load metadata and get site info
path = f"{HOME}/data/TrainingDFs/{region}/{output_res}M_Resolution/{region}_metadata.parquet"

meta = pd.read_parquet(path)
#reset index because we need cell_id as a column
meta.reset_index(inplace=True)

#set water year start/end dates based on ASO flights for the end date
aso_swe_files = [filename for filename in os.listdir(aso_swe_files_folder_path)]
aso_swe_files.sort()
print(aso_swe_files[0])
startyear = int(aso_swe_files[0][-16:-12])-1
startdate = f"{startyear}-09-30"

dates = []
for file in aso_swe_files:
    month = file[-12:-10]
    day = file[-10:-8]
    year = file[-16:-12]
    enddate = f"{str(year)}-{month}-{day}"
    dates.append(enddate)
dates.sort()
enddate = dates[-1]
enddate = pd.to_datetime(enddate)+pd.Timedelta(days=1)
enddate =enddate.strftime('%Y-%m-%d')

print(dates)

#get only water years with ASO observations
ASO_WYs = []
WYdict = {}
for year in WYs:
    try:
        WYdict[f"WY{year}"] = [d for d in dates if str(year) in d]
        WYdict[f"WY{year}"].sort()
        startdate_exception = f"{year-1}-10-01"
        enddate_exception =WYdict[f"WY{year}"][-1]
        ASO_WYs.append(year)
    except:
        print(f"No ASO observations for WY{year}")

print(ASO_WYs, startdate, enddate)

# Get bounding box for ASO image
min_lon = min(meta.cen_lon)*1.001
min_lat = min(meta.cen_lat)*.999
max_lon = max(meta.cen_lon)*.999
max_lat = max(meta.cen_lat)*1.001

print(min_lon, min_lat, max_lon, max_lat)

#Get aoi box
aoi_bbox = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

#Filter the ImageCollection by your Bounding Box
#precipitation product
hourly_precip = ee.ImageCollection(PrecipProduct).select(var).filterDate(startdate, enddate).filterBounds(aoi_bbox)
#hourly_precip = ee.ImageCollection(PrecipProduct).select(var).filterDate('2013-03-01', '2013-03-31')

collection_size = hourly_precip.size().getInfo()
print(f"Number of hourly precipitation images in the filtered collection: {collection_size}")

# Map the add_day_property function over the hourly collection
hourly_with_day_property = hourly_precip.map(add_day_property)

#Get a list of all unique days in the collection ---
# This creates a server-side List of unique day strings
unique_days = hourly_with_day_property.aggregate_array('day').distinct()
num_unique_days = unique_days.size().getInfo()
print(f"Number of unique days in the period: {num_unique_days}")

# Map daily precip function over the list of unique days
daily_sum_images_list = unique_days.map(sum_daily_precipitation)

# Convert the list of daily sum images into an ee.ImageCollection
daily_precipitation_collection = ee.ImageCollection(daily_sum_images_list)#.filterBounds(aoi_bbox)

# Get the size of the new daily collection
daily_count = daily_precipitation_collection.size().getInfo()
print(f"Number of images in the daily sum precipitation collection: {daily_count}")


ASO_USCATB_1000M_SWE_20130403.parquet
['2013-04-03', '2013-04-29', '2013-05-03', '2013-05-25', '2013-06-01', '2013-06-08']
[2013] 2012-09-30 2013-06-09
-119.91204691221265 37.70211392531725 -119.08804681848197 38.21903507431688
Number of hourly precipitation images in the filtered collection: 6048
Number of unique days in the period: 252
Number of images in the daily sum precipitation collection: 252


In [ ]:
hourly_precipitation = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select('total_precipitation')

In [64]:
hourly_precipitation = ee.ImageCollection(PrecipProduct).select(var)#.filterDate(startdate, enddate).filterBounds(aoi_bbox)
start_date = ee.Date('2013-03-01')
end_date = ee.Date('2013-03-05') # Example for a few days
num_days = end_date.difference(start_date, 'day').round()
days = ee.List.sequence(0, num_days.subtract(1)).map(lambda day_offset: start_date.advance(day_offset, 'day'))
daily_precipitation = ee.ImageCollection(days.map(lambda day:
    hourly_precipitation.filterDate(day, ee.Date(day).advance(1, 'day'))
    .sum()
    # .set('system:time_start', day.millis())
))

# # Chart daily precipitation for the point
# chart = ui.Chart.image.series(daily_precipitation, point, ee.Reducer.sum(), 30, 'system:time_start') \
#     .setOptions({
#         title: 'Daily Precipitation at Point',
#         vAxis: {title: 'Total Precipitation (m)'},
#         hAxis: {title: 'Date'}
#     })

# print(chart)

In [65]:
daily_precipitation

In [56]:
import ee
import pandas as pd
from datetime import date
from IPython.display import display
import matplotlib.pyplot as plt # For plotting the time series

# Initialize Earth Engine (ensure this is at the very top of your script)
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
    print("Earth Engine authenticated and initialized.")

# --- Define your Area of Interest (AOI) and Time Range ---
min_lon = -119.91204691221265
min_lat = 37.70211392531725
max_lon = -119.08804681848197
max_lat = 38.21903507431688
aoi_bbox = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

# Define your full project date range
start_date_str = '2012-10-01'
end_date_str = '2013-06-08'

PrecipProduct = 'NASA/NLDAS/FORA0125_H002'
var = 'total_precipitation'
scale = 12000 # NLDAS resolution in meters

# --- Re-create your daily_precipitation_collection (assuming your previous successful setup) ---
# This part is crucial and should come from your working script

hourly_precip = ee.ImageCollection(PrecipProduct).select(var).filterDate(start_date_str, end_date_str)

def add_day_property(image):
    date = ee.Date(image.get('system:time_start'))
    day_string = date.format('YYYY-MM-dd')
    return image.set('day', day_string)

hourly_with_day_property = hourly_precip.map(add_day_property)
unique_days = hourly_with_day_property.aggregate_array('day').distinct()

def sum_daily_precipitation(day_str):
    day_start = ee.Date(day_str)
    day_end = day_start.advance(1, 'day')
    daily_hourly_images = hourly_precip.filterDate(day_start, day_end)

    # Get a reference projection for consistent zero images
    try:
        ref_proj = hourly_precip.first().projection()
    except Exception:
        ref_proj = ee.Projection('EPSG:4326').atScale(scale)

    # Prepare a zero image
    zero_image = ee.Image.constant(0).rename('total_precipitation').unmask(0) \
                         .reproject(ref_proj) \
                         .clip(aoi_bbox)

    # Use ee.Algorithms.If to return sum or zero image
    daily_count = daily_hourly_images.size()
    result_image = ee.Algorithms.If(
        daily_count.eq(0),
        zero_image,
        daily_hourly_images.sum().clip(aoi_bbox).unmask(0)
    )
    result_image = ee.Image(result_image).set('system:time_start', day_start.millis()).set('day', day_str)
    return result_image

daily_sum_images_list = unique_days.map(sum_daily_precipitation)
daily_precipitation_collection = ee.ImageCollection(daily_sum_images_list).filterBounds(aoi_bbox)



In [ ]:
import matplotlib.pyplot as plt # Make sure you have this import

# ... (your code to create df_monthly_precip) ...

plt.figure(figsize=(12, 6)) # Create a figure for your plot
df_monthly_precip['mean_monthly_precip_mm'].plot(kind='bar') # This is the simple plotting command
plt.title(f'Mean Monthly Precipitation in AOI ({start_date_str} to {end_date_str})')
plt.xlabel('Month')
plt.ylabel('Mean Monthly Precipitation (mm)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show() # Display the plot

In [60]:
unique_months

In [61]:
# --- NEW: Sum daily precipitation to monthly totals ---

# 1. Get a list of unique months (YYYY-MM format)
def add_month_property(image):
    date = ee.Date(image.get('system:time_start'))
    month_string = date.format('YYYY-MM')
    return image.set('month', month_string)

# daily_precipitation_collection is used here as the base!
daily_with_month_property = daily_precipitation_collection.map(add_month_property)
unique_months = daily_with_month_property.aggregate_array('month').distinct()

# 2. Function to sum daily images for a given month
def sum_monthly_precipitation(month_str):
    month_start = ee.Date(ee.String(month_str).cat('-01')) 
    month_end = month_start.advance(1, 'month') # Exclusive end date for the month

    # Filter the DAILY collection for images within this month
    monthly_daily_images = daily_precipitation_collection.filterDate(month_start, month_end)

    # ... (rest of the logic to sum and handle empty months, as provided previously) ...
    if monthly_daily_images.size().eq(0):
        # ... (return zero image logic) ...
        # Ensure ref_proj here references a projection suitable for the daily collection
        try:
            ref_proj = daily_precipitation_collection.first().projection()
        except Exception:
            ref_proj = ee.Projection('EPSG:4326').atScale(12000) # Fallback

        return ee.Image.constant(0).rename('total_precipitation').unmask(0) \
                       .reproject(ref_proj) \
                       .clip(aoi_bbox) \
                       .set('system:time_start', month_start.millis()) \
                       .set('month', month_str)

    monthly_sum_image = monthly_daily_images.sum() # Summing the daily images
    
    # ... (clip, unmask, and set properties as before) ...
    clipped_monthly_sum_image = monthly_sum_image.clip(aoi_bbox)
    final_image = clipped_monthly_sum_image.unmask(0)

    return final_image.set('system:time_start', month_start.millis()) \
                      .set('month', month_str)


# Map the monthly sum function over the list of unique months
monthly_sum_images_list = unique_months.map(sum_monthly_precipitation)

# Convert the list of monthly sum images into an ee.ImageCollection
monthly_precipitation_collection = ee.ImageCollection(monthly_sum_images_list)

In [62]:
monthly_precipitation_collection

In [45]:
# --- Visualization Parameters for Daily Precipitation ---
# Daily precipitation values are typically lower than monthly sums.
daily_precip_vis = {
    'min': 0,
    'max': 50,  # Adjust this max based on what you expect for a single day's heavy rain (e.g., 20mm, 75mm)
    'palette': [
        'FFFFFF', 'CECECE', 'A0A0A0', '787878', '3C3C3C', # Greys for low/no precip
        '00FFFF', '00BFFF', '0080FF', '0000FF', '0000B0', # Blues for increasing precip
        '8A2BE2', '4B0082' # Purples for very high precip
    ]
}

# --- 1. Visualize a Specific Day from the Collection ---
# Let's pick a day in March 2013 that likely had precipitation
target_day_to_visualize = ee.Date('2013-03-20') # You can change this to any day in your collection

# Filter the daily collection to get the specific day's image
daily_image_to_display = daily_precipitation_collection \
    .filterDate(target_day_to_visualize, target_day_to_visualize.advance(1, 'day')) \
    .first() # Get the first (and only) image for that day

if daily_image_to_display:
    display_date = ee.Date(daily_image_to_display.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    
    # Create a Folium map centered on your AOI
    map_center = aoi_bbox.centroid().coordinates().getInfo()[::-1]
    m_daily_viz = folium.Map(location=map_center, zoom_start=10)

    # Add the AOI boundary to the map
    folium.GeoJson(
        aoi_bbox.getInfo(),
        name='Your AOI',
        style_function=lambda x: {'color': 'red', 'weight': 2, 'fillOpacity': 0}
    ).add_to(m_daily_viz)

    # Add the specific daily precipitation image to the map
    map_id_dict_daily = daily_image_to_display.getMapId(daily_precip_vis)
    folium.TileLayer(
        tiles=map_id_dict_daily['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        name=f'Daily Precip {display_date}'
    ).add_to(m_daily_viz)

    folium.LayerControl().add_to(m_daily_viz)
    print(f"\n--- Map of Daily Precipitation for {display_date} ---")
    display(m_daily_viz)
else:
    print(f"No daily image found for {target_day_to_visualize.format('YYYY-MM-dd').getInfo()} in the collection.")



--- Map of Daily Precipitation for 2013-03-20 ---


In [50]:
# --- 2. Visualize the Total Precipitation for the Entire Collection Period ---
# This sums all daily images in your collection to show total accumulated precipitation
print("\n--- Visualizing Total Precipitation for the Entire Collection Period ---")
total_period_precip = hourly_precip.sum().unmask(0)

# Adjust max for the full period sum (e.g., Oct 2012 - Jun 2013 could be 1000s of mm)
total_period_vis = {
    'min': 0,
    'max': 1500, # Use a higher max based on your expected total sum for the full period (e.g., ~1400mm for March alone)
    'palette': daily_precip_vis['palette'] # Reuse the same palette
}

m_total_period_viz = folium.Map(location=map_center, zoom_start=9) # Zoom out slightly for longer period overview

folium.GeoJson(
    aoi_bbox.getInfo(),
    name='Your AOI',
    style_function=lambda x: {'color': 'red', 'weight': 2, 'fillOpacity': 0}
).add_to(m_total_period_viz)

map_id_dict_total_period = total_period_precip.getMapId(total_period_vis)
folium.TileLayer(
    tiles=map_id_dict_total_period['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name=f'Total Precip {startdate}-{enddate}'
).add_to(m_total_period_viz)

folium.LayerControl().add_to(m_total_period_viz)
display(m_total_period_viz)


--- Visualizing Total Precipitation for the Entire Collection Period ---


In [52]:
total_period_precip['total_precipitation']

TypeError: 'Image' object is not subscriptable

In [ ]:
  #Get precipitation
        precip_poi = precip.getRegion(poi, output_res).getInfo()
        site_precip = EE_funcs.ee_array_to_df(precip_poi,['total_precipitation'])


In [11]:
dates = pd.date_range(start='2014-03-01', end='2014-04-01', freq='D')
dates

DatetimeIndex(['2014-03-01', '2014-03-02', '2014-03-03', '2014-03-04',
               '2014-03-05', '2014-03-06', '2014-03-07', '2014-03-08',
               '2014-03-09', '2014-03-10', '2014-03-11', '2014-03-12',
               '2014-03-13', '2014-03-14', '2014-03-15', '2014-03-16',
               '2014-03-17', '2014-03-18', '2014-03-19', '2014-03-20',
               '2014-03-21', '2014-03-22', '2014-03-23', '2014-03-24',
               '2014-03-25', '2014-03-26', '2014-03-27', '2014-03-28',
               '2014-03-29', '2014-03-30', '2014-03-31', '2014-04-01'],
              dtype='datetime64[ns]', freq='D')

In [24]:
import ee
import folium
from datetime import date
from IPython.display import display
import pandas as pd

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
    print("Earth Engine authenticated and initialized.")

# --- 1. Define your Area of Interest (AOI) and Time Range ---
min_lon = -119.91204691221265
min_lat = 37.70211392531725
max_lon = -119.08804681848197
max_lat = 38.21903507431688

aoi_bbox = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

start_date_str = '2012-10-01'
end_date_str = '2013-06-08'

d1 = date(2012, 10, 1)
d2 = date(2013, 6, 8)
num_days_in_period = (d2 - d1).days
print(f"Number of days in the specified period: {num_days_in_period}")

# --- 2. Load and Filter the NLDAS-2 Collection ---
# IMPORTANT CHANGE: Temporarily remove filterBounds(aoi_bbox) here for debugging.
# We will rely on .clip(aoi_bbox) inside the sum_daily_precipitation function.
nldas_hourly_collection = ee.ImageCollection('NASA/NLDAS/FORA0125_H002') \
    .filterDate(start_date_str, end_date_str) \
    .select('total_precipitation') # FilterBounds removed from here

hourly_count = nldas_hourly_collection.size().getInfo()
print(f"Number of hourly precipitation images in the filtered collection: {hourly_count}")
if hourly_count == 0:
    print("No hourly images found for the given criteria. Exiting.")
    exit()

# ========================================================================
# NEW DEBUGGING STEP: Inspect a raw hourly NLDAS image over a wider area
# ========================================================================
print("\n--- DEBUG: Inspecting a raw hourly NLDAS image over a wider region ---")
try:
    # Define a wider AOI for this test, covering more of California
    wider_min_lon = -124.0 # West coast
    wider_min_lat = 32.0 # Southern California
    wider_max_lon = -114.0 # East (Nevada border)
    wider_max_lat = 42.0 # Northern California
    wider_aoi_test_bbox = ee.Geometry.Rectangle([wider_min_lon, wider_min_lat, wider_max_lon, wider_max_lat])

    # Get a specific hourly image from the original NLDAS collection in March 2013
    hourly_test_date = '2013-03-15T12:00:00' # A specific hour on March 15th, 2013
    
    # Filter for just this hour globally (or over the wider test bbox if needed for performance on huge collections)
    single_hourly_image_raw = nldas_hourly_collection.filterDate(hourly_test_date, ee.Date(hourly_test_date).advance(1, 'hour')).first()
    
    if single_hourly_image_raw:
        # Get stats over the wider AOI for this raw hourly image
        hourly_image_stats_wider_test = single_hourly_image_raw.reduceRegion(
            reducer=ee.Reducer.minMax().unweighted(),
            geometry=wider_aoi_test_bbox, # Use the wider test AOI here
            scale=12000, # NLDAS resolution
            maxPixels=1e9
        ).getInfo()
        print(f"Raw Hourly NLDAS image stats for {hourly_test_date} over wider California AOI:")
        print(hourly_image_stats_wider_test)
        # IMPORTANT: Look at total_precipitation_max here. If it's 0, then NLDAS has no rain for that hour/region.

        # Visualize this raw hourly image
        hourly_raw_vis = {
            'min': 0,
            'max': 5, # Hourly precipitation values are usually small (mm)
            'palette': ['white', 'cyan', 'blue', 'darkblue'] # Simple palette to highlight any rain
        }
        map_center_for_hourly = wider_aoi_test_bbox.centroid().coordinates().getInfo()[::-1]
        m_hourly_raw = folium.Map(location=map_center_for_hourly, zoom_start=6) # Zoom out to see wider area

        folium.GeoJson(
            wider_aoi_test_bbox.getInfo(),
            name='Wider California AOI (Test)',
            style_function=lambda x: {'color': 'orange', 'weight': 2, 'fillOpacity': 0}
        ).add_to(m_hourly_raw)
        folium.GeoJson(
            aoi_bbox.getInfo(),
            name='Your Original AOI',
            style_function=lambda x: {'color': 'red', 'weight': 2, 'fillOpacity': 0}
        ).add_to(m_hourly_raw)

        map_id_dict_hourly_raw = single_hourly_image_raw.getMapId(hourly_raw_vis)
        folium.TileLayer(
            tiles=map_id_dict_hourly_raw['tile_fetcher'].url_format,
            attr='Google Earth Engine',
            overlay=True,
            name=f'Raw Hourly Precip {hourly_test_date}'
        ).add_to(m_hourly_raw)
        folium.LayerControl().add_to(m_hourly_raw)
        print(f"\n--- Map of Raw Hourly Precipitation for {hourly_test_date} ---")
        display(m_hourly_raw)
    else:
        print(f"Could not find a raw hourly image for {hourly_test_date}. Check date and collection availability.")
except Exception as e:
    print(f"Error during raw hourly image inspection: {e}")
# ========================================================================
# END NEW DEBUGGING STEP
# ========================================================================


# --- 3. Function to add a 'day' property to each image ---
def add_day_property(image):
    date = ee.Date(image.get('system:time_start'))
    day_string = date.format('YYYY-MM-dd')
    return image.set('day', day_string)

hourly_with_day_property = nldas_hourly_collection.map(add_day_property)

# --- 4. Get a list of all unique days in the collection ---
unique_days = hourly_with_day_property.aggregate_array('day').distinct()

num_unique_days = unique_days.size().getInfo()
print(f"Number of unique days in the period: {num_unique_days}")

# --- 5. Map over the unique days to create daily sums ---
def sum_daily_precipitation(day_str):
    day_start = ee.Date(day_str)
    day_end = day_start.advance(1, 'day')

    # Filter the hourly collection for this specific day
    daily_hourly_images = nldas_hourly_collection.filterDate(day_start, day_end)

    if daily_hourly_images.size().eq(0):
        # If no hourly images, create a masked image with 0 precipitation
        # Ensure it's reprojected and clipped correctly
        return ee.Image.constant(0).rename('total_precipitation').unmask(0) \
            .reproject(nldas_hourly_collection.first().projection()) \
            .clip(aoi_bbox) \
            .set('system:time_start', day_start.millis()) \
            .set('day', day_str)

    daily_sum_image = daily_hourly_images.sum()
    clipped_daily_sum_image = daily_sum_image.clip(aoi_bbox) # <--- CLIP IS STILL HERE

    # Ensure any masked pixels become 0
    final_image = clipped_daily_sum_image.unmask(0)

    return final_image.set('system:time_start', day_start.millis()) \
                                   .set('day', day_str)

daily_sum_images_list = unique_days.map(sum_daily_precipitation)
daily_precipitation_collection = ee.ImageCollection(daily_sum_images_list)

daily_count = daily_precipitation_collection.size().getInfo()
print(f"Number of images in the daily sum precipitation collection: {daily_count}")

# --- Get the bounds of the ImageCollection (this will now be global if filterBounds was removed) ---
collection_bounds = daily_precipitation_collection.geometry()
# Note: if the initial filterBounds was removed, this collection_bounds will be much larger.
# The clipping ensures your individual images are correct.
bounds_info = collection_bounds.bounds().getInfo()

all_lons = [point[0] for point in bounds_info['coordinates'][0]]
all_lats = [point[1] for point in bounds_info['coordinates'][0]]

bounds_min_lon = min(all_lons)
bounds_min_lat = min(all_lats)
bounds_max_lon = max(all_lons)
bounds_max_lat = max(all_lats)

print(f"\nImageCollection Bounds (might be wider now without initial filterBounds):")
print(f"  Min Lon: {bounds_min_lon}")
print(f"  Min Lat: {bounds_min_lat}")
print(f"  Max Lon: {bounds_max_lon}")
print(f"  Max Lat: {bounds_max_lat}")


# --- Debugging specific image precipitation values (from previous turns, still useful) ---
if daily_count > 0:
    first_day_image = ee.Image(daily_precipitation_collection.first())
    first_day_date = ee.Date(first_day_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

    print(f"\n--- DEBUG: Stats for first day ({first_day_date}) over your AOI ---")
    first_day_stats = first_day_image.reduceRegion(
       reducer=ee.Reducer.minMax().unweighted(),
        geometry=aoi_bbox,
        scale=12000, # NLDAS resolution
        maxPixels=1e9
    ).getInfo()
    print(first_day_stats)

    target_date_str_debug = '2013-03-01'
    ee_target_date_debug = ee.Date(target_date_str_debug)
    
    selected_day_collection_debug = daily_precipitation_collection.filterDate(
        ee_target_date_debug, ee_target_date_debug.advance(1, 'day')
    )

    if selected_day_collection_debug.size().getInfo() > 0:
        specific_day_image_debug = ee.Image(selected_day_collection_debug.first())
        print(f"\n--- DEBUG: Stats for specific day ({target_date_str_debug}) over your AOI ---")
        specific_day_stats = specific_day_image_debug.reduceRegion(
            reducer=ee.Reducer.minMax().unweighted(),
            geometry=aoi_bbox,
            scale=12000,
            maxPixels=1e9
        ).getInfo()
        print(specific_day_stats)
    else:
        print(f"\nDEBUG: No image found for {target_date_str_debug} in the collection.")

    # --- (Your code for getting first day's pixel values as a Pandas DataFrame) ---
    # ... (omitted for brevity, assume it's still here as in previous responses)

# --- (Your code for Daily Mean Extraction to DataFrame) ---
# ... (omitted for brevity, assume it's still here as in previous responses)

# --- (Your code for Visualization using Folium - original daily map and the March total map) ---
# ... (omitted for brevity, assume it's still here as in previous responses)

Number of days in the specified period: 250
Number of hourly precipitation images in the filtered collection: 6000

--- DEBUG: Inspecting a raw hourly NLDAS image over a wider region ---
Raw Hourly NLDAS image stats for 2013-03-15T12:00:00 over wider California AOI:
{'total_precipitation_max': 0, 'total_precipitation_min': 0}

--- Map of Raw Hourly Precipitation for 2013-03-15T12:00:00 ---


Number of unique days in the period: 250
Number of images in the daily sum precipitation collection: 250

ImageCollection Bounds (might be wider now without initial filterBounds):
  Min Lon: -119.91204691221267
  Min Lat: 37.702113925317235
  Max Lon: -119.08804681848198
  Max Lat: 38.2197550793228

--- DEBUG: Stats for first day (2012-10-01) over your AOI ---
{'total_precipitation_max': 0, 'total_precipitation_min': 0}

--- DEBUG: Stats for specific day (2013-03-01) over your AOI ---
{'total_precipitation_max': 0, 'total_precipitation_min': 0}


In [12]:
  for date in dates:  
    
    # 1. Define the specific date you want to select
    #    You can change this string to any date within your collection's range.
    target_date_str = date # <-- Set your desired date here!

    # 2. Convert the date string to an Earth Engine Date object
    ee_target_date = ee.Date(target_date_str)

    selected_day_collection = daily_precipitation_collection.filterDate(
        ee_target_date,
        ee_target_date.advance(1, 'day') # Advance 1 day from the target date
    )
    num_images_found = selected_day_collection.size().getInfo()

    if num_images_found > 0:
        # Since we designed daily_precipitation_collection to have one image per day,
        # .first() will give us that specific day's image.
        specific_day_image = ee.Image(selected_day_collection.first())
        actual_date_of_specific_image = ee.Date(specific_day_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

        print(f"\n--- Successfully selected image for: {actual_date_of_specific_image} ---")

        # Now you can use 'specific_day_image' for analysis, just like 'first_day_image'
        # For example, to get its pixel values into a DataFrame:

        sampling_scale = 12000 # meters

        sampled_pixels_specific_date = specific_day_image.sample(
            region=aoi_bbox,
            scale=sampling_scale,
            projection=specific_day_image.projection(),
            dropNulls=False,
            tileScale=16,
            geometries=True
        )

        sampled_data_specific_date_info = sampled_pixels_specific_date.getInfo()

        pixel_data_for_specific_df = []
        for feature in sampled_data_specific_date_info['features']:
            props = feature['properties']
            geom = feature['geometry']

            lon, lat = None, None
            if geom and geom['type'] == 'Point':
                lon, lat = geom['coordinates']

            pixel_data_for_specific_df.append({
                'longitude': lon,
                'latitude': lat,
                'precipitation_mm': props.get('total_precipitation')
            })

        df_specific_day_pixels = pd.DataFrame(pixel_data_for_specific_df)
        df_specific_day_pixels['precipitation_mm'] = df_specific_day_pixels['precipitation_mm'].fillna(0)
        df_specific_day_pixels.reset_index(drop=True, inplace=True)

       # print(f"\n--- Pixel Data (Head) for {actual_date_of_specific_image} ---")
       # print(df_specific_day_pixels.head())
        print(f"\nTotal pixels for {actual_date_of_specific_image} in AOI: {len(df_specific_day_pixels)}")

    else:
        print(f"\nNo image found for the target date: {target_date_str}. Please check the date range ({start_date_str} to {end_date_str}) and data availability.")

        
    if (df_specific_day_pixels['precipitation_mm'] > 1).any():
        display(df_specific_day_pixels)


--- Successfully selected image for: 2014-03-01 ---

Total pixels for 2014-03-01 in AOI: 54

--- Successfully selected image for: 2014-03-02 ---

Total pixels for 2014-03-02 in AOI: 54

--- Successfully selected image for: 2014-03-03 ---

Total pixels for 2014-03-03 in AOI: 54

--- Successfully selected image for: 2014-03-04 ---

Total pixels for 2014-03-04 in AOI: 54

--- Successfully selected image for: 2014-03-05 ---

Total pixels for 2014-03-05 in AOI: 54

--- Successfully selected image for: 2014-03-06 ---

Total pixels for 2014-03-06 in AOI: 54

--- Successfully selected image for: 2014-03-07 ---

Total pixels for 2014-03-07 in AOI: 54

--- Successfully selected image for: 2014-03-08 ---

Total pixels for 2014-03-08 in AOI: 54

--- Successfully selected image for: 2014-03-09 ---

Total pixels for 2014-03-09 in AOI: 54

--- Successfully selected image for: 2014-03-10 ---

Total pixels for 2014-03-10 in AOI: 54

--- Successfully selected image for: 2014-03-11 ---

Total pixels for

KeyboardInterrupt: 

In [6]:
sampled_pixels_specific_date

In [76]:
sampled_data_info

{'type': 'FeatureCollection',
 'columns': {'total_precipitation': 'MaskOnly'},
 'properties': {'band_order': ['total_precipitation']},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-119.88010288051873, 38.17829781202789]},
   'id': '0',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-119.77230504642439, 38.17829781202789]},
   'id': '1',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-119.66450721233005, 38.17829781202789]},
   'id': '2',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-119.5567093782357, 38.17829781202789]},
   'id': '3',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-119.44891154414135, 38.17829781202789]},
   'id': '

In [70]:
# --- Rest of your code (Extraction to DataFrame and Visualization) ---
reducer = ee.Reducer.mean()
scale = 12000 # meters

def get_aoi_precipitation(image):
    stats = image.reduceRegion(
        reducer=reducer,
        geometry=aoi_bbox,
        scale=scale,
        maxPixels=1e9
    )
    date = image.date()
    # Check if 'total_precipitation' key exists in stats before getting it
    # This can happen if reduceRegion result is empty (e.g., from a selfMasked image)
    precip_value = stats.get('total_precipitation')

    return ee.Feature(None, {
        'date': date.format('YYYY-MM-dd'),
        'timestamp': date.millis(),
        'mean_daily_precip_mm': precip_value
    })

daily_aoi_precipitation_feature_collection = daily_precipitation_collection.map(get_aoi_precipitation)

precipitation_data = daily_aoi_precipitation_feature_collection.getInfo()

parsed_data = []
for feature in precipitation_data['features']:
    props = feature['properties']
    if props['mean_daily_precip_mm'] is not None:
        parsed_data.append({
            'date': props['date'],
            'mean_daily_precip_mm': props['mean_daily_precip_mm']
        })

df_daily_precip = pd.DataFrame(parsed_data)
df_daily_precip['date'] = pd.to_datetime(df_daily_precip['date'])
df_daily_precip = df_daily_precip.sort_values(by='date').reset_index(drop=True)

print("\n--- Daily Mean Precipitation within AOI (First 5 Rows) ---")
print(df_daily_precip.head())

# --- Visualization (using Folium) ---
if daily_count > 0:
    first_daily_image = ee.Image(daily_precipitation_collection.first())
    daily_vis = {
        'min': 0,
        'max': 50,
        'palette': [
            'FFFFFF', 'CECECE', 'A0A0A0', '787878', '3C3C3C',
            '00FFFF', '00BFFF', '0080FF', '0000FF', '0000B0',
            '8A2BE2', '4B0082'
        ]
    }

    map_center_for_collection = collection_bounds.centroid().coordinates().getInfo()[::-1]
    m = folium.Map(location=map_center_for_collection, zoom_start=11)

    folium.GeoJson(
        collection_bounds.getInfo(),
        name='ImageCollection Bounds',
        style_function=lambda x: {'color': 'blue', 'weight': 3, 'fillOpacity': 0}
    ).add_to(m)

    folium.GeoJson(
        aoi_bbox.getInfo(),
        name='Original AOI Bounding Box',
        style_function=lambda x: {'color': 'FF0000', 'weight': 2, 'fillOpacity': 0}
    ).add_to(m)

    map_id_dict = first_daily_image.getMapId(daily_vis)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        name='First Daily Sum Precip'
    ).add_to(m)

    folium.LayerControl().add_to(m)
    display(m)

KeyError: 'date'

In [72]:
daily_aoi_precipitation_feature_collection

In [5]:
lat,long = meta[['cen_lat','cen_lon']].median()
location = ee.Geometry.Point(long,lat)

In [17]:
print(lat,long)

47.717357947477524 -123.52321734101645


In [33]:
startdate='2015-10'
enddate='2019-07'
precip = ee.ImageCollection('NASA/NLDAS/FORA0125_H002').select('total_precipitation').filterDate(startdate, enddate)

In [34]:
precip_poi = precip.getRegion(location,scale=1000).getInfo()

In [35]:
site_precip = EE_funcs.ee_array_to_df(precip_poi,['total_precipitation'])

In [36]:
temporal_resample = 'D'
kgm2_to_cm = 0.1

site_precip.set_index('datetime', inplace = True)
site_precip = site_precip.resample(temporal_resample).sum()
site_precip.reset_index(inplace = True)

        #make columns for cms
site_precip['total_precipitation'] = site_precip['total_precipitation']*kgm2_to_cm
site_precip.rename(columns={'total_precipitation':'daily_precipitation_cm'}, inplace = True)
site_precip.pop('time')
site_precip.set_index('datetime',inplace=True)


In [15]:
WYdict = {2016,2019}

,daily_precipitation_cm
datetime,
2017-09-10,0.06440
2017-09-11,0.00000
2017-09-12,0.00000
2017-09-13,0.00000
2017-09-14,0.00000
2017-09-15,0.00000
2017-09-16,0.00000
2017-09-17,0.85516
2017-09-18,0.81654


In [27]:
precip_daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('prcp').filterDate(startdate, enddate)
precip_daymet_poi = precip_daymet.getRegion(location,scale=1000).getInfo()

In [32]:
site_precip_daymet = EE_funcs.ee_array_to_df(precip_daymet_poi,['prcp'])

In [39]:
site_precip_daymet['prcp'] /= 10
site_precip_daymet.pop('time')
site_precip_daymet.set_index('datetime',inplace=True)
site_precip_daymet.rename(columns={'prcp':'daily_precipitation_cm'},inplace=True)

In [40]:
site_precip_daymet

,daily_precipitation_cm
datetime,
2015-10-01,0.00
2015-10-02,0.00
2015-10-03,0.00
2015-10-04,0.00
2015-10-05,0.00
...,...
2019-06-26,0.00
2019-06-27,0.63
2019-06-28,0.00


Map(center=[47.717357947477524, -123.52321734101645], controls=(WidgetControl(options=['position', 'transparen…

Map(center=[47.717357947477524, -123.52321734101645], controls=(WidgetControl(options=['position', 'transparen…